# 7 - Data Cleaning and Preparation

## Initial Imports

In [1]:
import pandas as pd
import numpy as np

In this chapter we discuss tools for handling missing data,
duplicate data, string manipulation, and some other analytical
data transformations. The next chapter is then focused on 
combining and rearranging datasets in various ways.

## 7.1 Handling Missing Data

Pandas has plenty of ways of handling missing data. Some of
the statistical built-in methods of pandas objects already 
exclude missing data for default, for example, but we may
be interested in handling missing data in different ways:

First, it is important we remember that we can set which 
values are considered NA when importing a dataset with 
`read_csv()` or other read functions by using the `na_values`
parameter.

Second, if the DataFrame is already loaded, we can treat it 
with functions such as `replace()` or `map()`.

### Filtering Out Missing Data

There are different ways of filtering missing data, depending
on whether we want to drop rows, columns, and the missing data
threshold we consider for being dropped.

Although we could use boolean indexing with the `notna()` method,
`dropna()` allows us to customize all these options above-mentioned.


In [3]:
import numpy as np
import pandas as pd

data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

Note that these return copies of the object by default. 
To modify the original dataset, we use the `inplace=` 
parameter.

To present some of the different ways to drop NA values:

In [2]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan], [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


`dropna()` drops rows that have any missing value by default:

In [3]:
data.dropna() 

,0,1,2
0,1.0,6.5,3.0


Passing `how='all'` drops only rows **that have all values
missing**

In [4]:
data.dropna(how='all') 

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


We can **drop columns** instead with the `axis` parameter:

In [6]:
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [7]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


To drop only past a certain **threshold** of missing values, 
we use the `thresh` parameter:

In [10]:
df = pd.DataFrame(np.random.standard_normal((7,3)))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

,0,1,2
0,0.254007,NaN,NaN
1,-0.249208,NaN,NaN
2,0.762543,NaN,-0.648712
3,-0.321158,NaN,1.165636
4,-0.749398,0.915997,-0.410406
5,-3.215473,0.348234,0.527618
6,0.609746,1.506953,-0.541533


In [11]:
df.dropna()

,0,1,2
4,-0.749398,0.915997,-0.410406
5,-3.215473,0.348234,0.527618
6,0.609746,1.506953,-0.541533


In [12]:
df.dropna(thresh=2)

,0,1,2
2,0.762543,NaN,-0.648712
3,-0.321158,NaN,1.165636
4,-0.749398,0.915997,-0.410406
5,-3.215473,0.348234,0.527618
6,0.609746,1.506953,-0.541533


### Filling In Missing Data

Rather than discarding missing data, we may
want to fill it with some value, such as an
integer, the mean for that columns, of the median.
`fillna()` will do that for us:

In [13]:
df.fillna(0) 

,0,1,2
0,0.254007,0.000000,0.000000
1,-0.249208,0.000000,0.000000
2,0.762543,0.000000,-0.648712
3,-0.321158,0.000000,1.165636
4,-0.749398,0.915997,-0.410406
5,-3.215473,0.348234,0.527618
6,0.609746,1.506953,-0.541533


To **use different fill values for different
columns**, we can pass a dictionary as parameter
to the method:

In [14]:
df.fillna({1:0, 2:2})

,0,1,2
0,0.254007,0.000000,2.000000
1,-0.249208,0.000000,2.000000
2,0.762543,0.000000,-0.648712
3,-0.321158,0.000000,1.165636
4,-0.749398,0.915997,-0.410406
5,-3.215473,0.348234,0.527618
6,0.609746,1.506953,-0.541533


We can also **fill forwards** or **fill backwards**
with the `ffill()` and `bfill()` methods:

In [15]:
df = pd.DataFrame(np.random.standard_normal((6,3)))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

,0,1,2
0,0.918578,-0.215545,1.107566
1,0.588271,-0.202194,1.014218
2,0.696783,NaN,0.077171
3,-1.907408,NaN,-0.298665
4,-0.770874,NaN,NaN
5,-0.503009,NaN,NaN


In [17]:
df.ffill()

,0,1,2
0,0.918578,-0.215545,1.107566
1,0.588271,-0.202194,1.014218
2,0.696783,-0.202194,0.077171
3,-1.907408,-0.202194,-0.298665
4,-0.770874,-0.202194,-0.298665
5,-0.503009,-0.202194,-0.298665


In [18]:
df.ffill(limit=2)

,0,1,2
0,0.918578,-0.215545,1.107566
1,0.588271,-0.202194,1.014218
2,0.696783,-0.202194,0.077171
3,-1.907408,-0.202194,-0.298665
4,-0.770874,NaN,-0.298665
5,-0.503009,NaN,-0.298665


With `fillna()` we may also fill with the mean
or median of a column:

In [19]:
df.fillna(df.mean())

,0,1,2
0,0.918578,-0.215545,1.107566
1,0.588271,-0.202194,1.014218
2,0.696783,-0.208870,0.077171
3,-1.907408,-0.208870,-0.298665
4,-0.770874,-0.208870,0.475072
5,-0.503009,-0.208870,0.475072


## 7.2 Data Transformation

Asides from dealing with missing data, filtering, cleaning and
transforming are also essential parts of the data wrangling job:

### Removing duplicates

Consider the following example of a DataFrame that contains duplicates:

In [4]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'], 'k2': [1, 1, 2, 3, 3, 4, 4,]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The `duplicated()` method returns a boolean array indicating 
if any row is a duplicate of a previously iterated row:

In [5]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

The `drop_duplicates()` method returns a DataFrame with only
the rows indicated as `False` by `duplicated()`:

In [6]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


These methods by default consider all columns, but 
suppose we want to restrict the duplicate checking 
and dropping to only a subset of columns. We do that
with the `subset` parameter:

In [7]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [8]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6    False
dtype: bool

In [9]:
data.duplicated(subset=['k1'])

0    False
1    False
2     True
3     True
4     True
5     True
6     True
dtype: bool

By default, `drop_duplicates()` keeps the first values
it encounters. Passing `keep='last'` will keep the last
ones instead.

In [10]:
data.drop_duplicates(subset=['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function Or Mapping

Frequently we'll want to do some transformation depending
on the values present in the current array. Consider this
hypothetical data collected about kinds of meat:

In [11]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon", "pastrami", "corned beef", "bacon", "pastrami", "honey ham", "nova lox"], "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose we want to add a new column indicating the 
type of animal the meat came from. We can create a 
dict to map each meat to an animal:

In [12]:
meat_to_animal = {
  'bacon' : 'pig',
  'pulled pork': 'pig',
  'pastrami' : 'cow',
  'corned beef' : 'cow',
  'honey ham' : 'pig',
  'nova lox' : 'salmon'
}

The `map` method accepts a function or a dictionary-like
object to perform a function or mapping into a series of
values:

In [13]:
data['animal'] = data['food'].map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


Passing a function that returns the value from the dict
would also have done the trick. Lets try it with a lambda
function:

In [15]:
data['food'].map(lambda x: meat_to_animal[x])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values

While `map()` can be seen as a way to replace values,
`replace()` offers a simpler and more flexible way to 
do so. Consider the Series:

In [16]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The `-999` values might be a sentinel for missing data.
To replace them with a value that pandas understands as 
NA, we can use `replace()`:

In [17]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

We can also **replace multiple values at once** with 
by passing a list of values to the first argument:

In [18]:
values_to_replace = [-999, -1000]
data.replace(values_to_replace, np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

**Replacing different values with different
replacements** can be done by passing a list
to the second argument (of equal length to the
first list) or by passing a dict as argument:

In [19]:
data.replace({-999: 0, -1000: np.nan})

0    1.0
1    0.0
2    2.0
3    0.0
4    NaN
5    3.0
dtype: float64

### Renaming Axis Indexes

We can both create new objects with different labels
than the first one, or modify labels in place.
To modify it in place, we can use the `index.map()` 
method assigning it to the index of the DataFrame:

In [21]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [23]:
data.index = data.index.map(lambda x: x[:4].upper())
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


To return a new object without modifying the original, 
we use the `rename()` method:

In [25]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Rename can also be used with a dictionary-like object
to rename specific values to specific replacements:

In [28]:
data.rename(index={'OHIO': 'Indiana'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
Indiana,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

We can organize continuous data into discrete
groups (or bins) for analysis. Suppose this data
to be categorized in different bins:

In [1]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

To divide it into bins, we first determine the
limits of our bins and then use `pd.cut` with
our data and bins as parameters:

In [4]:
bins = [18, 25, 35, 60, 100]

age_categories = pd.cut(ages, bins)

age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

This is a special Categorical object. Each bin
is identified by this special pandas interval value
containing upper and lower limits. Let's explore this 
object:

In [5]:
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [6]:
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [7]:
age_categories.categories[0]

Interval(18, 25, closed='right')

In [9]:
age_categories.value_counts()

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

Here, the interval openness is indicated by the
parenthesis and brackets. The parenthesis means the
interval is open (exclusive) and the bracket means 
closed (inclusive). We can change the side that's
inclusive with the `right=False` parameter:

In [12]:
pd.cut(ages, bins, right=False).value_counts()

[18, 25)     4
[25, 35)     4
[35, 60)     3
[60, 100)    1
Name: count, dtype: int64

We may with to use other names for labels instead
of the interval-based ones with the `labels` parameter:

In [14]:
age_groups = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=age_groups)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If we pass an integer as the bins argument, pandas
will automatically create equally spaced intervals
to bin our data based on the maximum and minimum 
values:

In [15]:
data = np.random.uniform(size=20)
pd.cut(data, 4, precision=2) # the precision limits the precision of the decimal points of the bins to two digits

[(0.75, 0.96], (0.33, 0.54], (0.54, 0.75], (0.33, 0.54], (0.11, 0.33], ..., (0.75, 0.96], (0.75, 0.96], (0.11, 0.33], (0.75, 0.96], (0.54, 0.75]]
Length: 20
Categories (4, interval[float64, right]): [(0.11, 0.33] < (0.33, 0.54] < (0.54, 0.75] < (0.75, 0.96]]

A closely related function is `pd.qcut()`, which 
will separate the data in equally populated quantiles:

In [19]:
data = np.random.standard_normal(size=1000)
quartiles = pd.qcut(data, 4, precision=2)
quartiles

[(0.75, 2.78], (0.75, 2.78], (0.75, 2.78], (0.00089, 0.75], (0.00089, 0.75], ..., (0.00089, 0.75], (-0.63, 0.00089], (0.00089, 0.75], (0.75, 2.78], (-3.17, -0.63]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.17, -0.63] < (-0.63, 0.00089] < (0.00089, 0.75] < (0.75, 2.78]]

In [20]:
quartiles.value_counts()

(-3.17, -0.63]      250
(-0.63, 0.00089]    250
(0.00089, 0.75]     250
(0.75, 2.78]        250
Name: count, dtype: int64

We can also pass our own intervals for the quantiles,
as long as they are number between 0 and 1, inclusive:

In [22]:
pd.qcut(data, [0, 0.1, 0.5, 0.7, 1]).value_counts()

(-3.1559999999999997, -1.229]    100
(-1.229, 0.000888]               400
(0.000888, 0.556]                200
(0.556, 2.777]                   300
Name: count, dtype: int64

### Detecting and Filtering Outliers

Filtering or transforming outliers is usually a matter
of applying array operations. Consider this DataFrame
with some normally distributed data:

In [23]:
data = pd.DataFrame(np.random.standard_normal((1000, 4)))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.006178,-0.009601,0.015320,0.036232
std,1.049986,0.951804,0.978321,1.024074
min,-3.389640,-3.138652,-3.057713,-3.109405
25%,-0.691853,-0.681501,-0.593412,-0.663168
50%,-0.032227,-0.010748,0.014075,-0.011084
75%,0.681431,0.614412,0.662042,0.703315
max,3.849684,2.633940,3.055968,3.444042


Suppose we want to find all values in a given column
with absolute value > 3:

In [25]:
col = data[2]
col[col.abs() >3]

229    3.048082
320   -3.057713
988    3.055968
Name: 2, dtype: float64

To find all rows with any absolute value bigger than
three, we can use the `any` method on a Boolean DataFrame:

In [27]:
data[(data.abs()>3).any(axis='columns')]

,0,1,2,3
161,0.559674,2.332783,-0.758124,-3.104536
229,-1.745371,0.678667,3.048082,-0.621166
320,-1.485553,-1.033165,-3.057713,-0.746016
524,-3.389640,-0.359639,-0.717857,0.259953
595,-1.160640,0.411437,1.513938,3.047130
612,-0.338480,0.472264,-1.059129,-3.109405
743,0.013883,-3.138652,0.417210,0.516035
786,-0.251886,0.095524,-0.749549,3.073178
879,3.849684,2.141543,-0.741813,0.140411
883,-0.053997,1.027895,-0.267932,-3.008369


We can use the boolean indexing of `data.abs() > 3`
to set a new value, for example, 3 or -3. Here, 
we'll use the `np.sign()` method to determine the 
sign of the data, and therefore if it should have
3 or -3 attributed:

In [32]:
data[data.abs() > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.006638,-0.009463,0.015274,0.035890
std,1.046015,0.951357,0.977821,1.021665
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.691853,-0.681501,-0.593412,-0.663168
50%,-0.032227,-0.010748,0.014075,-0.011084
75%,0.681431,0.614412,0.662042,0.703315
max,3.000000,2.633940,3.000000,3.000000


### Permutation and Random Sampling

So, there's this numpy method that returns a permutation
the size we give it in the parameter:

In [39]:
sampler = np.random.permutation(5)
sampler

array([3, 1, 2, 0, 4])

We can use this with `iloc[]` to return a random sample
of rows. We can also use it with the `take()` method,
that accepts an `axis` argument to sample columns as well.

In [33]:
df = pd.DataFrame(np.arange(5*7).reshape(5, 7))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [40]:
df.iloc[sampler]

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
0,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34


In [41]:
df.take(sampler, axis=1)

,3,1,2,0,4
0,3,1,2,0,4
1,10,8,9,7,11
2,17,15,16,14,18
3,24,22,23,21,25
4,31,29,30,28,32


Alternatively, there's the `sample()` df method,
which also allows for `replace=True` parameter
to allow for replacements:

In [42]:
df.sample(n=3, axis=1, replace=True)

,2,2,0
0,2,2,0
1,9,9,7
2,16,16,14
3,23,23,21
4,30,30,28


### Computing Indicator/Dummy Variables

A dummy or indicator matrix is a matrix indicating
if a value belongs to a certain category or not.
Take the example:

In [43]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], "data1": range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [44]:
pd.get_dummies(df['key'])

,a,b,c
0,False,True,False
1,False,True,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,True,False


We can also add a prefix to our dummy columns
to make it easier to join them to the original 
DataFrame later:

In [46]:
pd.get_dummies(df['key'], prefix='key')

,key_a,key_b,key_c
0,False,True,False
1,False,True,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,True,False


If a row in a dataframe belongs to multiple categories,
we'll have to use a different approach to create the 
dummies, using the `str.get_dummies(<separator>)` method:

In [48]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('../pydata-book/datasets/movielens/movies.dat', sep='::', 
                       header=None, names=mnames, engine='python')
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [50]:
dummies = movies['genres'].str.get_dummies('|')
dummies.iloc[:10, :6]

,Action,Adventure,Animation,Children's,Comedy,Crime
0,0,0,1,1,1,0
1,0,1,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
5,1,0,0,0,0,1
6,0,0,0,0,1,0
7,0,1,0,1,0,0
8,1,0,0,0,0,0
9,1,1,0,0,0,0


Then we could add a prefix with the `add_prefix()` method:

In [52]:
dummies.add_prefix('genre_')

,genre_Action,genre_Adventure,genre_Animation,genre_Children's,genre_Comedy,genre_Crime,genre_Documentary,genre_Drama,genre_Fantasy,genre_Film-Noir,genre_Horror,genre_Musical,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Thriller,genre_War,genre_Western
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3879,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3880,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3881,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


A useful statistical application is to use `get_dummies()` with
binning to create a matrix indicating belonging to a certain bin:

In [54]:
np.random.seed(12345)
values=np.random.uniform(size=10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [55]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,True
1,False,True,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,False,False,True,False,False
5,False,False,True,False,False
6,False,False,False,False,True
7,False,False,False,True,False
8,False,False,False,True,False
9,False,False,False,True,False


### Summary

We've learned plenty of data transformation techniques:

- Removing and checking for duplicates with `duplicated()` and `drop_duplicates()`
- Transforming data with `.map` and a function or dictionary-like object;
- Replacing values with `replace()` and lists of values or dict-like objects;
- Renaming Axis Indexes with `rename()` and `index=`, `column=` parameters;
- Discretization and Binning with `pd.cut` and `pd.qcut()`, accepting different quantiles;
- Detecting and Filtering outliers with assigning with boolean arrays and `np.sign(data)`;
- Permutation and Random Sampling with `np.random.permutation(n)` and `iloc[]` or `take()`,
plus `df.sample(n, axis, replace)`;
- Computing dummies and indicators with `pd.get_dummies(col)` and `df[col].str.get_dummies(sep)`
- Combining dummies with binnings with `pd.get_dumies(pd.cut(values, bins))`;

## 7.3 Extension Data Types

Consider the following integer array:

In [2]:
s = pd.Series([1, 4, 2, None])
s

0    1.0
1    4.0
2    2.0
3    NaN
dtype: float64

The float64 type comes from the fact that pandas
uses `np.nan` as a NA sentinel. We can force another
dtype by using the `dtype` parameter in the series 
creation:

In [4]:
s = pd.Series([1, 4, 2, None], dtype=pd.Int64Dtype())
s

0       1
1       4
2       2
3    <NA>
dtype: Int64

This uses the sentinel value `pd.NA`:

In [5]:
s[3] is pd.NA

True

We can also declare the type with a shorter string:

In [7]:
s = pd.Series([1, 4, 2, None], dtype='Int64')
s

0       1
1       4
2       2
3    <NA>
dtype: Int64

There is also a `StringDType`, but it requires the 
pyarrow library:

In [8]:
s = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())
s

0      one
1      two
2     <NA>
3    three
dtype: string

We can convert dtypes with the `astype()` method
as part of the data cleaning process:

In [9]:
df = pd.DataFrame({"A": [1, 2, None, 4], "B": ["one", "two", "three", None], "C": [False, None, False, True]})
df

,A,B,C
0,1.0,one,False
1,2.0,two,None
2,NaN,three,False
3,4.0,None,True


In [11]:
df['A'] = df['A'].astype('Int64')
df['B'] = df['B'].astype('string')
df['C'] = df['C'].astype('boolean')
df

,A,B,C
0,1,one,False
1,2,two,<NA>
2,<NA>,three,False
3,4,<NA>,True


## 7.4 String Manipulation

We use built-in string methods along with regular
expressions for string manipulation, as python's 
built-in methods are sufficient for a variety of tasks:

### Python Built-in String Object Methods

We can split using a delimiter:

In [12]:
val = 'a,b,     guido'
val.split(',')

['a', 'b', '     guido']

We can combine that with `trim()` for removing whitespaces:

In [13]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

We can concat these strings with another delimiter
using the `.join()` method:

In [14]:
'::'.join(pieces)

'a::b::guido'

We can locate substrings with the `in` keyword and the
`index()` and `find()` methods:

In [15]:
'gui' in val

True

In [17]:
val.index('gui')

9

In [18]:
val.find('mel')

-1

We can count substrings:

In [20]:
val.count(',')

2

And replace them too:

In [21]:
val.replace(',', '::')

'a::b::     guido'

Replace can be used to remove occurrences too:

In [22]:
val.replace(',', '')

'ab     guido'

### Regular Expressions

Regular expressions are a dense topic, but here
what's important is that they allow us to find
complex patterns within strings and string datasets.

To use regex, we import the `re` library:

In [23]:
import re

In [26]:
text = 'foo       bar\t baz     \tqux'
re.split(r'\s+', text)

['foo', 'bar', 'baz', 'qux']

Here, we split using any number of whitespaces with
the `\s+` regex. When we call this expression, it is
first compiled and then applies. We can compile the 
regex first with `re.compile`, which is better suited
for repeated use of the regex:

In [27]:
regex = re.compile(r'\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

To instead of splitting, **find all occurrences** of 
a pattern, we can use `.findall()` regex method:

In [28]:
regex.findall(text)

['       ', '\t ', '     \t']

`search()` and `match()` are closely related to `findall`,
but search only returns the first match and match 
only matches at the beginning of the string. Consider the following 
block of text and regex pattern:

In [29]:
text = """Dave dave@google.com 
Steve steve@gmail.com 
Rob rob@gmail.com 
Ryan ryan@yahoo.com"""
pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"

regex = re.compile(pattern, flags=re.IGNORECASE)

re.IGNORECASE makes the regex ignore case sensitivity.

Using `findall` here produces a list of emails:

In [30]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

Search will return a special object for the first 
email found:

In [31]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [32]:
text[m.start():m.end()]

'dave@google.com'

Match won't return anything, as the pattern isn't 
found in the beginning of the string:

In [34]:
print(regex.match(text))

None


The `sub` method substitutes the pattern everywhere
it is found:

In [36]:
print(regex.sub('REDACTED', text))

Dave REDACTED 
Steve REDACTED 
Rob REDACTED 
Ryan REDACTED


If we want the resulting pattern to be return in separate
parts, as in parts of a tuple, we can add parenthesis to 
each pattern part: 

In [37]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
regex = re.compile(pattern, flags=re.IGNORECASE)

A match object will be split with this in its `groups()` method:

In [39]:
m = regex.match('fake_email43@gmail.com')
m.groups()

('fake_email43', 'gmail', 'com')

`findall()` returns a list of tuples:

In [40]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

The `sub()` method also has access to each of the values
in the groups created. `\1` refers to the first value, `\2`
to the second, and so on and so forth:

In [43]:
print(regex.sub(r'Username: \1, domain: \2, Suffix: \3', text))

Dave Username: dave, domain: google, Suffix: com 
Steve Username: steve, domain: gmail, Suffix: com 
Rob Username: rob, domain: gmail, Suffix: com 
Ryan Username: ryan, domain: yahoo, Suffix: com


### String Functions in pandas

Pandas has specialized string functions for dealing with
data in this format. Consider this list of emails in a 
series:

In [44]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com", "Rob": "rob@gmail.com", "Wes": np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

The fact that this data has missing values makes using 
map methods and functions harder. Hence, string methods
such as contains:

In [46]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

To make correct typing of this Series, we may use the
extension type concept learned earlier:

In [47]:
data_str = data.astype('string')
data_str.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

Regular expressions can also be use, along with many
`re` options such as `IGNORECASE`:

In [48]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

We can use both indexing or `.get` method to retrieve
from this:

In [54]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [57]:
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

We can also slice with this syntax:

In [60]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

The `extract()` method returns the groups of a 
regex as a DataFrame:

In [61]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


This part of the book presents a table with a list of 
useful string methods. Refer to it in page 235 or in
Pandas documentation.

## 7.5 Categorical Data

When using categorical methods, we store the values
not as their strings, but as integer-coding and 
category-corresponding pairs. This makes calculations
and storage more efficient. Consider the following 
example: 

In [63]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

N = len(fruits)

rng = np.random.default_rng(seed=12345)

df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count':rng.integers(3, 15, size=N),
                   'weight': rng.uniform(0,4, size=N)},
                   columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


Here, `df['fruit']` is an array of strings with quite
some repetition. We could convert it to categorical by 
calling:

In [64]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

The values for `fruit_cat` are now an instance of 
`pd.Categorical`, which can be accessed by the 
`.array` attribute:

In [65]:
c = fruit_cat.array
type(c)

pandas.core.arrays.categorical.Categorical

It has `codes` and `categories` attributes:

In [66]:
fruit_cat.array.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [67]:
c.categories

Index(['apple', 'orange'], dtype='object')

To get a mapping between codes and categories, we can
use this trick:

In [69]:
dict(enumerate(c.categories)) # enumerate returns tuples with indices and values

{0: 'apple', 1: 'orange'}

To convert a DataFrame column to categorical, just assign the 
result to it:

In [70]:
df['fruit'] = df['fruit'].astype('category')
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

We can also create a `pandas.Categorical` from 
other sequences:

In [71]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

If we've obtained the categories encoded from another 
source, we can use the `.from_codes` constructor:

In [72]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

Regarding **ordered** categorical data (ordinal), we
may explicitly say the data is ordered when constructing,
or convert it to ordered:

In [73]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [74]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

As a last note, any Immutable data type can be made
into Categorical, not just strings.

### Computations with Categoricals

Categoricals are better optimized for functions
like `groupby` and `agg`, which are routinely
used in data analysis. Also, they use less 
memory than their non-categorical counterparts.

As an example of performance, consider a `value_counts()`
of this 10 million dataset, as well as its memory usage:

In [3]:
N = 10_000_000
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N//4))

In [5]:
categories = labels.astype('category')

In [8]:
lab_m = labels.memory_usage(deep=True)

In [9]:
cat_m = categories.memory_usage(deep=True)

In [10]:
cat_m/lab_m

0.01923174896424834

In [15]:
%timeit labels.value_counts()

245 ms ± 6.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit categories.value_counts()

31.6 ms ± 858 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Categorical Methods

Categories have special `.cat.methods()` as well, 
specially but not limited to:

- `set_categories()`
- `remove_unused_categories()`

Finally, it is important to remember that we can transform
categories into *dummy variables* with the `pd.get_dummies()`
method, which will come in handy for preparing data for 
machine learning algorithms.

## Summary (7.3 onwards:)

Today we've learned:

- To load and transform pandas data with missing values with `dtype=` and `astype()`
- To set up regex with `re(pattern)`, to compile them, to look for patterns to:
    - `split()` at pattern; 
    - `findall()` instances of pattern;
    - `search()` for first occurrence of pattern;
    - `match()` at string start; 
    - Return the result of `findall()` in multiple parts by wrapping the pattern in parenthesis;
    - To sub each piece of the pattern with `\1', \2`...
- To use pandas `str` functions to work with string data, such as:
    - `contains()`;
    - `findall(regex, flags)`;
    - `slice` with `str[:n]`
    - create groups from regex findall pieces with `str.extract(pattern)`
- To turn columns into `pandas.Categorical` for performance;
- To get the categories-codes with `dict(enumerate(df[column].array.categories))`
- To create ordered categoricals from constructor `ordered=True` or conversion `.as_ordered()`
- Some categorical methods such as `cat.set_categories` and `cat.remove_unused_categories`

